In [ ]:
# @title Instalar dependencias

r = "Retrieval"
b = "based"
v = "Voice"
c = "Conversion"
w = "WebUI"

!apt install -q aria2
!wget -q https://raw.githubusercontent.com/fumiama/{"-".join([r,b,v,c,w])}/e1aeb16630158cd6b0b9db8d6d27fe1ad869e42d/requirements/py311.txt
!pip install -q -r py311.txt
!rm *.txt

!mkdir /content/rvc
%cd /content/rvc
!git config --global init.defaultBranch main
!git init && git remote add origin https://github.com/fumiama/{"-".join([r,b,v,c,w])}.git
!git fetch origin e1aeb16630158cd6b0b9db8d6d27fe1ad869e42d --depth=1 && git reset --hard FETCH_HEAD

# Descargar modelos pre-entrenados
aria2c = 'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M'
%cd assets
hugging_user = "".join(["l", "j", "1995"])
huggingface_repo = f"https://huggingface.co/{hugging_user}/{v + c + w}/resolve/main/"
!{aria2c} {huggingface_repo}hubert_base.pt -d hubert -o hubert_base.pt
!{aria2c} {huggingface_repo}rmvpe.pt -d rmvpe -o rmvpe.pt
!{aria2c} {huggingface_repo}uvr5_weights/onnx_dereverb_By_FoxJoy/vocals.onnx -d uvr5_weights/onnx_dereverb_By_FoxJoy -o vocals.onnx
for model in ['D', 'G', 'f0D', 'f0G']:
  !{aria2c} {huggingface_repo}pretrained_v2/{model}48k.pth -d pretrained_v2 -o {model}48k.pth
for model in [
        "HP2-%E4%BA%BA%E5%A3%B0vocals%2B%E9%9D%9E%E4%BA%BA%E5%A3%B0instrumentals",
        "HP2_all_vocals",
        "HP3_all_vocals",
        "HP5-%E4%B8%BB%E6%97%8B%E5%BE%8B%E4%BA%BA%E5%A3%B0vocals%2B%E5%85%B6%E4%BB%96instrumentals",
        "HP5_only_main_vocal",
        "VR-DeEchoAggressive",
        "VR-DeEchoDeReverb",
        "VR-DeEchoNormal",
]:
  !{aria2c} {huggingface_repo}uvr5_weights/{model}.pth -d uvr5_weights -o {model}.pth
%cd ..

!rm -r /content/sample_data

from IPython.display import clear_output
clear_output(wait=True)
print("\n---------------------------\n| Dependencias instaladas |\n---------------------------\n")

In [ ]:
# @title Conectar con Google Drive (Opcional)

from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# @title Descargar archivo/carpeta de google drive por URL (Opcional)

DESCARGAR_PARA = "clips_audio" # @param ["clips_audio", "modelo_para_entrenar", "modelo_entrenado"]

match (DESCARGAR_PARA):
  case "clips_audio":
    !mkdir /content/dataset/
    %cd /content/dataset
  case "modelo_para_entrenar":
    %cd logs
  case "modelo_entrenado":
    %cd assets/weights

URL = "https://drive.google.com/drive/folders/1mx2ZjK3hrKHmj8RbAVkaehBVSx7Hie0M" # @param {type:"string"}
URL = URL.strip().removeprefix("https://").removeprefix("http://")
if URL.startswith("drive.google.com/"):
  if URL.startswith("drive.google.com/file/d/"):
    URL = URL.removeprefix("drive.google.com/file/d/").split("/")[0]
    !ls > /tmp/oldfiles.txt
    !gdown "{URL}"
    !ls > /tmp/newfiles.txt
    !diff /tmp/oldfiles.txt /tmp/newfiles.txt | tail -1 | sed -e "s/> //" > /tmp/newfile.txt
    !7z x $(cat /tmp/newfile.txt) && rm $(cat /tmp/newfile.txt)
  elif URL.startswith("drive.google.com/drive/folders/"):
    URL = URL.removeprefix("drive.google.com/drive/folders/").split("/")[0]
    !gdown --folder "{URL}"
  else:
    !gdown "{URL}"

if DESCARGAR_PARA == "clips_audio":
  !find -type f -exec mv {} ./ \;
  !find . -empty -type d -delete

%cd /content/rvc

In [ ]:
# @title Configurar modelo
from torch import cuda

NOMBRE_DEL_MODELO = "xadii"  # @param {type:"string"}

# aliases de variables de español a ingles
MODELNAME = NOMBRE_DEL_MODELO

DATASET_DIR = f"logs/{MODELNAME}"
![ -d {DATASET_DIR} ] || mkdir -p {DATASET_DIR}
!echo BATCHSIZE recomendado: {int((cuda.get_device_properties(0).total_memory / 1024 / 1024 / 1024 + 0.4) // 2)}

In [ ]:
# @title Procesar clips de audio
from multiprocessing import cpu_count

DIRECTORIO_CLIPS_AUDIO = "/content/dataset" # @param {type: "string"}
ALGORITMO = "rmvpe"  # @param [ "pm", "dio", "harvest", "rmvpe"]
PROCESAR_EN_PARALELO = False  # @param {type: "boolean"}
PREPROCESS_PER = "3.7" # @param [ "3.0", "3.7" ]
SAMPLE_RATE = 48000
IS_HALF = True # @param {"type": "boolean"}

# aliases de variables de español a ingles
NOPARALLEL = not PROCESAR_EN_PARALELO
PREPROCESS_PER = float(PREPROCESS_PER)
THREADCOUNT = cpu_count()

!python infer/modules/train/preprocess.py "{DIRECTORIO_CLIPS_AUDIO}" {SAMPLE_RATE} {THREADCOUNT} {DATASET_DIR} {NOPARALLEL} {PREPROCESS_PER}
!python infer/modules/train/extract_f0_print.py {DATASET_DIR} {THREADCOUNT} {ALGORITMO} "cuda" {IS_HALF}
!python infer/modules/train/extract_feature_print.py "cuda" 1 0 0 {DATASET_DIR} "v2" {IS_HALF}

In [ ]:
# @title Entrenar modelo

NUMERO_DE_PASOS = 100 # @param {type:"integer"}
# @markdown Cada cuantos pasos se deberia guardar el modelo en disco en caso de fallos:
GUARDAR_PASOS = 20 # @param {type:"integer"}
BATCHSIZE = 8  # @param {type:"slider", min:1, max:32, step:1}
# @markdown Guardar cache del entrenamiento en vram (solo recomendado para entrenamientos con menos de 10 minutos de audio)
CACHE_EN_VRAM = False  # @param {type:"boolean"}
# @markdown Solamente guardar el ultimo ckpt para ahorrar espacio en disco
ULTIMO_CKPT = True  # @param {type:"boolean"}

# aliases de variables de español a ingles
MODELEPOCH = NUMERO_DE_PASOS
EPOCHSAVE = GUARDAR_PASOS

# convertir variables del form para ser leibles por los scripts
MODELSAMPLE = "48k"
CACHEDATA = int(CACHE_EN_VRAM)
ONLYLATEST = int(ULTIMO_CKPT)

# Crear archivos necesarios para el entrenamiento
!cat configs/v2/48k.json | sed -e 's/"log_interval": 200,/"log_interval": {EPOCHSAVE},/' | sed -e 's/"epochs": 20000,/"epochs": {MODELEPOCH},/' | sed -e 's/"batch_size": 4,/"batch_size": {BATCHSIZE},/' > {DATASET_DIR}/config.json
from os import listdir
gt_wavs_dir = f"{DATASET_DIR}/0_gt_wavs"
feature_dir = f"{DATASET_DIR}/3_feature768"
f0_dir = f"{DATASET_DIR}/2a_f0"
f0nsf_dir = f"{DATASET_DIR}/2b-f0nsf"

names = (
  set([name.split(".")[0] for name in listdir(gt_wavs_dir)])
  & set([name.split(".")[0] for name in listdir(feature_dir)])
  & set([name.split(".")[0] for name in listdir(f0_dir)])
  & set([name.split(".")[0] for name in listdir(f0nsf_dir)])
)
opt = []
for name in names:
  opt.append(f"{gt_wavs_dir}/{name}.wav|{feature_dir}/{name}.npy|{f0_dir}/{name}.wav.npy|{f0nsf_dir}/{name}.wav.npy|0")
for _ in range(2):
  opt.append("logs/mute/0_gt_wavs/mute48k.wav|logs/mute/3_feature768/mute.npy|logs/mute/2a_f0/mute.wav.npy|logs/mute/2b-f0nsf/mute.wav.npy|0")
with open("/tmp/filelist.txt", "w") as f:
  f.write("\n".join(opt))
!cat /tmp/filelist.txt | shuf > {DATASET_DIR}/filelist.txt
!rm /tmp/filelist.txt

!python infer/modules/train/train.py -e {MODELNAME} -sr {MODELSAMPLE} -f0 1 -bs {BATCHSIZE} -g 0 -te {MODELEPOCH} -se {EPOCHSAVE} -pg assets/pretrained_v2/f0G{MODELSAMPLE}.pth -pd assets/pretrained_v2/f0D{MODELSAMPLE}.pth -l {ONLYLATEST} -c {CACHEDATA} -v "v2"